In [1]:
# load packages
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from typing import List, Optional

In [2]:
# 경로 설정
data_dir = '/Users/jaewone/Downloads/floor_noise/sensor_data/dataset'

noise_data_dir = os.path.join(data_dir, 'noise')
white_data_dir = os.path.join(data_dir, 'white')

### 데이터 가져오기


CSV 파일을 읽어 데이터를 가져온다. 이때 모델을 아두이노에 올릴 수 있도록 데이터를 -128~127 사이의 정수로 변환한다.


In [16]:
def quantized_data(data: np.ndarray) -> np.ndarray:
    """
    값을 0과 1사이로 정규화한 다음 -128에서 127 사이의 int8 값으로 양자화한다.
    """
    data[-1] = -data[-1]
    SENSOR_MAX = 1.0
    SENSOR_MIN = -2.0

    # normalized_values = (data - SENSOR_MIN) / \
    #     (SENSOR_MAX - SENSOR_MIN) * 2.0 - 1.0
    # return np.clip(np.round(normalized_values * 127), -128, 127).astype(np.int8)
    return np.clip(data*64, -128, 127).astype(np.int8)

# BLE white: 44, 40, -41
# IOT white: 42, 36, -41
# BLE Noise: 41, 47, -45


# file_path = '/Users/jaewone/Downloads/floor_noise/sensor_data/dataset/white/white_121.csv'  # 42, 36, -41
file_path = '/Users/jaewone/Downloads/floor_noise/sensor_data/dataset/noise/noise_121.csv'  # 41, 47, -45
# file_path = '/Users/jaewone/Downloads/floor_noise/test_sample_data.txt'  # 현장에서 측정한 white
# df = pd.read_csv(os.path.join(noise_data_dir, file_path))
df = pd.read_csv(file_path)
df.apply(quantized_data, axis=1).apply(
    lambda x: print(f'{x[0]}, {x[1]}, {x[2]}'), axis=1)

0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 66
0, 3, 65
0, 3, 65
0, 3, 66
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 66
0, 3, 65
0, 3, 65
0, 3, 66
0, 3, 64
0, 3, 65
0, 3, 66
0, 3, 64
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 66
0, 3, 64
0, 3, 65
0, 3, 66
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 66
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 65
0, 3, 66
0

0      None
1      None
2      None
3      None
4      None
       ... 
123    None
124    None
125    None
126    None
127    None
Length: 128, dtype: object

In [ ]:
def quantized_data(data: np.ndarray) -> np.ndarray:
    """
    값을 0과 1사이로 정규화한 다음 -128에서 127 사이의 int8 값으로 양자화한다.
    """
    # SENSOR_MAX = 1.0
    # SENSOR_MIN = -2.0

    # normalized_values = (data - SENSOR_MIN) / \
    #     (SENSOR_MAX - SENSOR_MIN) * 2.0 - 1.0
    # return np.clip(np.round(normalized_values * 127), -128, 127).astype(np.int8)
    return np.clip((data*64), -128, 127).astype(np.int8)


def get_data(noise_data_dir: str, white_data_dir: str, rate_ratio=(0.7, 0.2, 0.1), quantized=True) -> List[np.ndarray]:
    assert abs(sum(rate_ratio) - 1) < 1e-9, 'sum of rate_ratio should be 1'
    assert len(rate_ratio) == 3, 'length of rate_ratio should be 3'

    # load noise data
    noise_data = []
    for file in os.listdir(noise_data_dir):
        noise_data.append(pd.read_csv(
            os.path.join(noise_data_dir, file)))
    noise_data = np.array(noise_data)
    print(f'noise data: {noise_data.shape}')

    # set noise label
    # noise_label = np.ones(noise_data.shape[0])
    noise_label = np.array([(1, 0)]*noise_data.shape[0])

    # load white data
    white_data = []
    for file in os.listdir(white_data_dir):
        white_data.append(pd.read_csv(
            os.path.join(white_data_dir, file)))
    white_data = np.array(white_data)
    print(f'white data: {white_data.shape}')

    # set white label
    # white_label = np.zeros(white_data.shape[0])
    white_label = np.array([(0, 1)]*white_data.shape[0])

    # merge data
    data = np.concatenate((noise_data, white_data))
    if quantized:
        data = quantized_data(data)
    label = np.concatenate((noise_label, white_label))
    print(f'total data: {data.shape}\n')

    # add dimension
    data = data.reshape(*data.shape, 1)

    # split train, validation, test data
    train_data, test_data, train_label, test_label = train_test_split(
        data, label, test_size=rate_ratio[2], random_state=42)
    train_data, val_data, train_label, val_label = train_test_split(
        train_data, train_label, test_size=rate_ratio[1]/(rate_ratio[0]+rate_ratio[1]), random_state=42)

    # check splited rate
    print(
        f'train data: {train_data.shape}   Rate: {round(train_data.shape[0]/data.shape[0], 2)}')
    print(
        f'val data:   {val_data.shape}   Rate: {round(val_data.shape[0]/data.shape[0], 2)}')
    print(
        f'test data:  {test_data.shape}    Rate: {round(test_data.shape[0]/data.shape[0], 2)}')
    return train_data, train_label, val_data, val_label, test_data, test_label


train_data, train_label, val_data, val_label, test_data, test_label = get_data(
    noise_data_dir, white_data_dir)
train_data.shape, train_label.shape, val_data.shape, val_label.shape, test_data.shape, test_label.shape

In [ ]:
# 데이터가 양자화되었는지 확인
train_data[0][0]

### 모델 구성


In [ ]:
# 모델 구성
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(8, (3, 4), activation='relu', input_shape=(
        128, 3, 1), padding='same'),  # (batch, 128, 3, 8)
    tf.keras.layers.MaxPool2D((3, 3), padding='same'),  # (batch, 43, 1, 8)
    tf.keras.layers.Conv2D(16, (3, 4), activation='relu',
                           padding='same'),  # (batch, 43, 1, 16)
    tf.keras.layers.MaxPool2D((3, 3), padding='same'),  # (batch, 15, 1, 16)
    tf.keras.layers.Flatten(),  # (batch, 240)
    tf.keras.layers.Dense(16, activation="relu"),  # (batch, 16)
    tf.keras.layers.Dense(2, activation='softmax'),  # (batch, 2)
])
model.summary()

### 모델 학습


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=['accuracy'])

# 4. 모델 학습
model.fit(train_data, train_label, epochs=10, batch_size=32,
          validation_data=(val_data, val_label))

### 예측 및 평가


In [ ]:
np.round(model.predict(test_data)[:10], 2)

In [ ]:
# predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

y_pred_max = np.argmax(model.predict(test_data), axis=1)
y_test_max = np.argmax(test_label, axis=1)
y_pred_max_sum = np.sum(y_pred_max)

confusion_matrix(y_test_max, y_pred_max)
print('accuracy: ', np.sum(y_test_max == y_pred_max) / len(y_test_max))
print('precision: ', np.sum(y_test_max * y_pred_max == 1) /
      y_pred_max_sum if y_pred_max_sum != 0 else 0)
print('recall: ', np.sum(y_test_max * y_pred_max == 1) / np.sum(y_test_max))
print('f1-score: ', f1_score(y_test_max, y_pred_max))

### tflite로 저장


In [ ]:
# save tflite model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
shape = (128, 3, 1)


# def representative_data_gen():
#     for _ in range(100):
#         data = np.random.rand(1, *shape)
#         yield [data.astype(np.float32)]


# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.representative_dataset = representative_data_gen
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# converter.inference_input_type = tf.int8
# converter.inference_output_type = tf.float32

tflite_model = converter.convert()
open("model.tflite", "wb").write(tflite_model)

In [ ]:
# 저장된 tflite 모델 확인
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_details, output_details

In [ ]:
from tqdm import tqdm

correct = 0
incorrect = 0

for i in tqdm(range(len(test_data))):
    input_data = np.array([test_data[i]], dtype=np.float32)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])[0]
    if np.argmax(output_data) == np.argmax(test_label[i]):
        correct += 1
    else:
        incorrect += 1

# 정확도
print(f'Accuracy: {correct/(correct+incorrect)}')

In [ ]:
import random

# 저장된 tflite 파일 테스트
for i in [random.randint(0, len(test_data)-1) for _ in range(5)]:
    input_shape = input_details[0]['shape']
    input_data = np.array([test_data[i]], dtype=np.float32)
    # print(f'input shape: {input_data.shape}')
    print(f'label: {test_label[i]}')

    interpreter.set_tensor(input_details[0]['index'], input_data)

    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    print(f'pred:  {output_data[0]}')
    print()

### 바이트 파일로 변환


In [ ]:
# C source file를 만들기 위한 xxd 라이브러리 설치
# !apt-get -qq install xxd

# 모델을 C 소스 파일로 변경한다.
!xxd -i model.tflite > model.cc

# C 소스 파일을 출력한다.
!cat model.cc